In [ ]:
import bw2data as bd
import bw2io as bi
import bw2regional as bwr

Here if our matrix equation:

$ h_{r} = [ MN_{dx}DXN_{g}GR ]^{T} \cdot [ BA^{-1}f ]$

In [ ]:
bd.projects.set_current("LC IMPACT case study")

In [ ]:
meal = bd.Database('meal')
meal.process()

In [ ]:
my_method = ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'All')

In [ ]:
meal.metadata['geocollections'] = ['BCN', 'ALM']
bd.databases.flush()

In [ ]:
import time

Data for the creation of the **spatial units** in the third spatial scale.

In [ ]:
job = bwr.remote.calculate_intersection(
    'ALM', 
    'watersheds-eq-sw-all'
)

while job.status == 'starting':
    time.sleep(1)

Add the results of the intersection calculation

In [ ]:
bwr.remote.intersection(
    'BCN', 
    'watersheds-eq-sw-all'
)

This is the data in the intersection:

In [ ]:
bwr.Intersection(('ALM', 'watersheds-eq-sw-all')).load()

Create a new geocollection, **and** add data for the $D$ and $G$ matrices.

In [ ]:
bwr.remote.intersection_as_new_geocollection(
    'ALM', 
    'watersheds-eq-sw-all', 
    'ALM - watersheds-eq-sw-all'
)

In [ ]:
'ALM - watersheds-eq-sw-all' in bwr.geocollections

Calculate the data that will go in the $X$ matrix.

In [ ]:
job = bwr.remote.calculate_rasterstats(
    'ALM - watersheds-eq-sw-all', 
    'weighted-pop-density'
)

In [ ]:
while job.status == 'started':
    time.sleep(1)
    
job.status

In [ ]:
bwr.remote.rasterstats_as_xt(    
    'ALM - watersheds-eq-sw-all', 
    'weighted-pop-density',
    'this is silly there is only one raster cell',
)

In [ ]:
missing = [
    ('ecoinvent', 'ALM - watersheds-eq-sw-all'),
    ('BCN', 'ALM - watersheds-eq-sw-all'),
    ('world', 'ALM - watersheds-eq-sw-all'),
    ('ALM - watersheds-eq-sw-all', 'world'),
]

In [ ]:
for x, y in missing:
    try:
        bwr.remote.calculate_intersection(x, y)
    except:
        pass

In [ ]:
for x, y in missing:
    bwr.remote.intersection(x, y)

In [ ]:
lca = bwr.ExtensionTablesLCA(
    demand={('meal', 'meal'): 1},
    method=my_method,
    xtable='this is silly there is only one raster cell'
)
lca.lci()
lca.lcia()

In [ ]:
lca.geodataframe_xtable_spatial_scale().explore(column='score_rel')